Make BGP-LLaMa and Mobile LLaMa web services.

With the web service, develop the real user cases for the network
operation and administrators.
For example, I think the real-time monitoring function and the
generation report in the natural language.
- IP prefix comparison from different regional bgp sources
- Specific AS report: AS18298 (CNU) BGP report for last 5 minutes
- Hourly/daily BGP monitoring summarization/report with plots
  * We can directly use 5-minute near real-time data of  bgp routing
tables and update messages.
- BGP anomaly detection reports

In [ ]:
import pybgpstream
stream = pybgpstream.BGPStream(
    # accessing ris-live
    project="ris-live",
    # filter to show only stream from rrc00
    filter="collector rrc00",
)

for elem in stream:
    print(elem)

In [ ]:
import pybgpstream
stream = pybgpstream.BGPStream(
    # accessing routeview-stream
    project="routeviews-stream",
    # filter to show only stream from amsix bmp stream
    filter="router amsix",
)

for elem in stream:
    print(elem)

In [ ]:
import pybgpstream

# Create a BGPStream instance
stream = pybgpstream.BGPStream(
    project="ris-live",
    record_type="updates",
)

# Start the live stream
for rec in stream.records():
    print(f"Time: {rec.time}")
    for elem in rec:
        print(f"Peer ASN: {elem.peer_asn}, Peer IP: {elem.peer_address}, Type: {elem.type}")
        if elem.type in ('A', 'W'):
            print(f"Prefix: {elem.fields['prefix']}, AS Path: {elem.fields.get('as-path', 'N/A')}")
        print("-" * 40)

In [12]:
import pybgpstream
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Initialize pandas DataFrames for prefix announcements and withdrawals
prefix_announcements_df = pd.DataFrame(columns=['Time', 'Prefix', 'Count'])
withdrawals_df = pd.DataFrame(columns=['Time', 'Prefix', 'Count'])

# Create a BGPStream instance
stream = pybgpstream.BGPStream(
    project="ris-live",
    record_type="updates",
)

# Collect data over a defined period
collection_period = timedelta(minutes=5)  # Collect data for 5 minutes
start_time = datetime.utcnow()

# Start the live stream
for rec in stream.records():
    current_time = datetime.utcfromtimestamp(rec.time)
    rounded_time = current_time.replace(second=0, microsecond=0)

    # Stop collecting data after the defined period
    if rounded_time >= start_time + collection_period:
        print("Generating a report")
        break

    for elem in rec:
        prefix = elem.fields['prefix']
        if elem.type == 'A':
            if prefix in prefix_announcements_df['Prefix'].values:
                prefix_announcements_df.loc[prefix_announcements_df['Prefix'] == prefix, 'Count'] += 1
            else:
                new_row = pd.DataFrame({'Time': [current_time], 'Prefix': [elem.fields['prefix']], 'Count': [1]})
                prefix_announcements_df = pd.concat([prefix_announcements_df, new_row], ignore_index=True)
        elif elem.type == 'W':
            if prefix in withdrawals_df['Prefix'].values:
                withdrawals_df.loc[withdrawals_df['Prefix'] == prefix, 'Count'] += 1
            else:
                new_row = pd.DataFrame({'Time': [current_time], 'Prefix': [elem.fields['prefix']], 'Count': [1]})
                withdrawals_df = pd.concat([withdrawals_df, new_row], ignore_index=True)

# Function to plot data from DataFrame
def plot_data(df, title):
    if not df.empty:
        grouped_df = df.groupby('Prefix').sum().reset_index()
        plt.figure(figsize=(10, 5))
        plt.bar(grouped_df['Prefix'], grouped_df['Count'], align='center')
        plt.xticks(rotation=45)
        plt.title(title)
        plt.show()
    else:
        print(f"No data to display for {title}")

# Plot the collected data
plot_data(prefix_announcements_df, "Prefix Announcements (Collected Data)")
plot_data(withdrawals_df, "Withdrawals (Collected Data)")

1701406265 HTTP ERROR: Transferred a partial file (18)


In [11]:
prefix_announcements_df

,Time,Prefix,Count
0,2023-12-01 04:31:18.840,2804:4498:8000::/36,512
1,2023-12-01 04:31:18.840,67.211.53.0/24,704
2,2023-12-01 04:31:18.840,2400:a560::/34,989
3,2023-12-01 04:31:18.840,2400:a560:c000::/34,989
4,2023-12-01 04:31:18.840,2400:a560:8000::/34,989
...,...,...,...
41872,2023-12-01 04:31:26.860,204.196.89.0/24,1
41873,2023-12-01 04:31:26.860,204.196.7.0/24,1
41874,2023-12-01 04:31:26.860,204.196.188.0/24,1
41875,2023-12-01 04:31:26.860,204.196.202.0/24,1


In [ ]:
import pybgpstream
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time

def plot_data(df, title):
    if not df.empty:
        grouped_df = df.groupby('Prefix').sum().reset_index()
        plt.figure(figsize=(10, 5))
        plt.bar(grouped_df['Prefix'], grouped_df['Count'], align='center')
        plt.xticks(rotation=45)
        plt.title(title)
        plt.show()
    else:
        print(f"No data to display for {title}")

def collect_and_plot_data():
    # Create a BGPStream instance
    stream = pybgpstream.BGPStream(
        project="ris-live",
        record_type="updates",
    )

    # Initialize pandas DataFrames for prefix announcements and withdrawals
    prefix_announcements_df = pd.DataFrame(columns=['Time', 'Prefix', 'Count'])
    withdrawals_df = pd.DataFrame(columns=['Time', 'Prefix', 'Count'])

    # Collect data over a defined period
    collection_period = timedelta(minutes=5)  # Collect data for 5 minutes
    start_time = datetime.utcnow()

    # Start the live stream
    for rec in stream.records():
        current_time = datetime.utcfromtimestamp(rec.time)

        # Stop collecting data after the defined period
        if current_time >= start_time + collection_period:
            break

        for elem in rec:
            if elem.type == 'A':
                new_row = {'Time': current_time, 'Prefix': elem.fields['prefix'], 'Count': 1}
                prefix_announcements_df = prefix_announcements_df.append(new_row, ignore_index=True)
            elif elem.type == 'W':
                new_row = {'Time': current_time, 'Prefix': elem.fields['prefix'], 'Count': 1}
                withdrawals_df = withdrawals_df.append(new_row, ignore_index=True)

    # Plot the collected data
    plot_data(prefix_announcements_df, "Prefix Announcements (Collected Data)")
    plot_data(withdrawals_df, "Withdrawals (Collected Data)")

# Continuous loop
while True:
    collect_and_plot_data()
    # Add a delay if needed, for example, wait for 1 minute before starting the next collection period
    time.sleep(60)  # Sleep for 60 seconds


# IP prefix comparison from different regional bgp sources

In [ ]:
import pybgpstream
from datetime import datetime
from collections import defaultdict

# Define the prefixes and collectors
prefixes_to_compare = ["192.0.2.0/24", "203.0.113.0/24"]  # Example prefixes
collectors = ["route-views.linx", "route-views.sydney", "route-views.sao"]

# Define the time window for data collection
start_time = "2023-01-01 00:00:00 UTC"
end_time = "2023-01-02 00:00:00 UTC"

# Collect data
prefix_data = defaultdict(lambda: defaultdict(list))
stream = pybgpstream.BGPStream(
    from_time=start_time,
    until_time=end_time,
    collectors=collectors,
    record_type="updates",
    filter=f"prefix {' or '.join(prefixes_to_compare)}"
)

for rec in stream.records():
    if rec.status == "valid":
        collector = rec.collector
        for elem in rec:
            if elem.type == "A" and elem.fields["prefix"] in prefixes_to_compare:
                as_path = elem.fields["as-path"]
                prefix_data[elem.fields["prefix"]][collector].append(as_path)

# Compare data
for prefix, data in prefix_data.items():
    print(f"Comparing for prefix {prefix}:")
    for collector, paths in data.items():
        print(f"  Collector {collector} announced paths: {paths}")
    print("-" * 40)
